In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
import geopandas as gp
from shapely import wkt
from shapely.geometry import LineString

In [ ]:
plt.style.use('asu-light')

In [ ]:
base_pop = pd.read_csv('../model_inputs/base/persons.csv')
scenario_pop = pd.read_csv('../model_inputs/npv_low_opcost/persons.csv')

In [ ]:
base_flows = pd.read_csv('../data/congestion_results/base_am_flows.csv')
npv_flows = pd.read_csv('../data/congestion_results/npv_am_flows.csv')
graph = pd.read_csv('../data/congestion_results/base_am_flows.graph.csv')  # both base and npv use same graph

In [ ]:
base_flows = base_flows.merge(graph, left_on=['segment_id', 'target_segment_id'], right_on=['source_id', 'target_id'], validate='1:1')
npv_flows = npv_flows.merge(graph, left_on=['segment_id', 'target_segment_id'], right_on=['source_id', 'target_id'], validate='1:1')

In [ ]:
base_flows['freeflow_tt'] = base_flows.freeflow_traversal_time_secs + base_flows.turn_costs
base_flows['congested_tt_ratio'] = base_flows.congested_travel_time_secs / base_flows.freeflow_tt
npv_flows['freeflow_tt'] = npv_flows.freeflow_traversal_time_secs + npv_flows.turn_costs
npv_flows['congested_tt_ratio'] = npv_flows.congested_travel_time_secs / npv_flows.freeflow_tt

In [ ]:
# b/c of how the SCAG cost functions work, motorway links may have congested travel time < freeflow travel time. Don't count those as negative
# time in congestion
base_time_in_congestion = np.sum(np.maximum(base_flows.congested_travel_time_secs - base_flows.freeflow_traversal_time_secs, 0) * base_flows.turn_flow) / 60 / len(base_pop)
npv_time_in_congestion = np.sum(np.maximum(npv_flows.congested_travel_time_secs - npv_flows.freeflow_traversal_time_secs, 0) * npv_flows.turn_flow) / 60 / len(base_pop)
print(f"""
{base_time_in_congestion=}
{npv_time_in_congestion=}
""")

## Congestion maps

In [ ]:
base_flows.head()

In [ ]:
# every edge with the same source represents the same road segment. combine them so they don't all sit on top of one another
grpd = base_flows.groupby('segment_id')
base_segments = pd.DataFrame({
    'congested_tt_ratio': grpd.congested_tt_ratio.mean(),
    'congested_travel_time_secs': grpd.congested_travel_time_secs.mean(),
    'freeflow_tt': grpd.freeflow_tt.mean(),
    'length_m': grpd.length_m.first(),
    'rclass': grpd.rclass.first(),
    'geom': grpd.geom.first()
})
base_segments['geom'] = base_segments.geom.map(wkt.loads)
base_segments = gp.GeoDataFrame(base_segments, geometry='geom')
base_segments.crs = {'init': 'epsg:4326'}

In [ ]:
base_segments = base_segments.to_crs(epsg=26911)  # so we can use simple math

In [ ]:
def heading (pt1, pt2):
    return np.arctan2((pt2[0] - pt1[0]), (pt2[1] - pt1[1]))

In [ ]:
# https://github.com/numpy/numpy/issues/5228
def pol2cart(theta, rho):
    x = rho * np.cos(theta + np.pi / 2)
    y = rho * np.sin(theta + np.pi / 2)
    return x, y

In [ ]:
def offset_geom_to_right(geom, distance=100):
    if len(geom.coords) == 0:
        return geom
    else:
        output_coords = []
        # offset first coord 90 degrees right
        # nb all calcs here in radians
        initial_hdg = heading(geom.coords[0], geom.coords[1])
        offset_hdg = initial_hdg + np.pi / 2
        offx, offy = pol2cart(offset_hdg, distance)
        output_coords.append([geom.coords[0][0] + offx, geom.coords[0][1] + offy])
        
        for prev, cur, nxt in zip(geom.coords[:-2], geom.coords[1:-1], geom.coords[2:]):
            hdg = heading(prev, nxt)
            offset_hdg = hdg + np.pi / 2
            offx, offy = pol2cart(offset_hdg, distance)
            output_coords.append([cur[0] + offx, cur[1] + offy])
            
        # and the final coord
        final_hdg = heading(geom.coords[-2], geom.coords[-1])
        offset_hdg = final_hdg + np.pi / 2
        offx, offy = pol2cart(offset_hdg, distance)
        output_coords.append([geom.coords[-1][0] + offx, geom.coords[-1][1] + offy])
    
        return LineString(output_coords)

In [ ]:
base_segments['geom'] = base_segments.geom.map(offset_geom_to_right)

In [ ]:
base_segments['congest_cat'] = pd.cut(base_segments.congested_tt_ratio, [0, 1.1, 1.25, 1.5, 2, np.inf], labels=['≤10%', '10%–25%', '25%–50%', '50%–100%', '>100%'])

In [ ]:
land = gp.read_file('../../sorting/data/ne_10m_land.shp').to_crs(epsg=26911)

In [ ]:
f, ax = plt.subplots(figsize=(16, 12))
land.plot(color='#fafafa', ax=ax)
cong_col = {
    '≤10%': '#00a3e0',
    '10%–25%': '#fef0d9',
    '25%–50%': '#fdcc8a',
    '50%–100%': '#fc8d59',
    '>100%': '#d7301f'
    
}
base_segments.plot(ax=ax, legend=False, lw=1, color=base_segments.congest_cat.replace(cong_col))

plt.xlim(345_000, 500_000)
plt.ylim(3_700_000, 3_810_000)
plt.xticks([])
plt.yticks([])

patches = [mpatch.Patch(color=c) for c in cong_col.values()]
labels = cong_col.keys()


ax.legend(
    patches,
    labels,
    loc='upper right',
    title='Travel speed vs. freeflow',
    framealpha=1,
    fontsize='medium',
    title_fontsize='medium'
)
plt.text(1, 0, 'Colors courtesy ColorBrewer', transform=ax.transAxes, ha='right', va='bottom')

# label some things
plt.text(460_000, 3_780_000, 'Inland Empire')
plt.text(420_000, 3_720_000, 'Orange County')
plt.text(381_000, 3_771_000, 'Central Los Angeles')

plt.savefig('../../dissertation/fig/abm/congestion_base.png', dpi=600, bbox_inches='tight')

## Now change in congestion

In [ ]:
base_segments['scenario_tt'] = npv_flows.groupby('segment_id').congested_travel_time_secs.mean().reindex(base_segments.index)
assert not base_segments.scenario_tt.isnull().any()

In [ ]:
base_segments['delta_tt'] = base_segments.scenario_tt / base_segments.congested_travel_time_secs - 1
base_segments.delta_tt.describe()

In [ ]:
base_segments['delta_cat'] = pd.cut(base_segments.delta_tt, [-np.inf, -0.25, -0.05, 0.05, 0.25, np.inf], labels=['Decrease ≥25%', 'Decrease 5%–25%', 'Within 5%', 'Increase 5%–25%', 'Increase >25%'])

In [ ]:
f, ax = plt.subplots(figsize=(16, 12))
land.plot(color='#fafafa', ax=ax)
cong_col = {
'Increase >25%': '#a6611a',
    'Increase 5%–25%': '#dfc27d',
    'Within 5%': '#dddddd',
    'Decrease 5%–25%': '#014B34',
    'Decrease ≥25%': '#016644'
}

#dfc27d
#f5f5f5
#80cdc1
#018571

base_segments.plot(ax=ax, legend=False, lw=1, color=base_segments.delta_cat.replace(cong_col))

plt.xlim(345_000, 500_000)
plt.ylim(3_700_000, 3_810_000)
plt.xticks([])
plt.yticks([])

patches = [mpatch.Patch(color=c) for c in cong_col.values()]
labels = cong_col.keys()


ax.legend(
    patches,
    labels,
    loc='upper right',
    title='Change in travel time',
    framealpha=1,
    fontsize='medium',
    title_fontsize='medium'
)
plt.text(1, 0, 'Colors courtesy ColorBrewer', transform=ax.transAxes, ha='right', va='bottom')

# label some things
plt.text(460_000, 3_780_000, 'Inland Empire')
plt.text(420_000, 3_720_000, 'Orange County')
plt.text(381_000, 3_763_000, 'Central Los Angeles')

plt.savefig('../../dissertation/fig/abm/congestion_delta.png', dpi=600, bbox_inches='tight')

In [ ]:
plt.hist(base_segments.congested_travel_time_secs / base_segments.freeflow_tt, weights=base_segments.length_m / base_segments.length_m.sum(), bins=np.arange(0, 3, 0.125), alpha=0.5, label="Base")
plt.hist(base_segments.scenario_tt / base_segments.freeflow_tt, weights=base_segments.length_m / base_segments.length_m.sum(), bins=np.arange(0, 3, 0.125), alpha=0.5, label="Low operating cost")
plt.xlim(0.75, 3)
plt.yticks([])
plt.xlabel("Ratio of congested to free-flow travel time")
plt.ylabel("Kilometers of roadway")
plt.legend()
plt.savefig('../../dissertation/fig/abm/congestion_hist.pdf', bbox_inches='tight')

In [ ]:
base_segments.loc[base_segments.delta_tt < 0, 'rclass'].value_counts()

In [ ]:
base_segments.loc[base_segments.rclass == 'motorway_link', 'delta_tt'].describe()

In [ ]:
test_gdf = base_segments.iloc[[1, 1]].copy()

In [ ]:
test_gdf.geom.iloc[1] = offset_geom_to_right(test_gdf.geom.iloc[0], 100)

In [ ]:
test_gdf['idx'] = [1, 2]

In [ ]:
test_gdf.plot(column='idx', legend=True)

In [ ]:
test_gdf.geom.iloc[1].coords[0]

In [ ]:
heading([1, 1], [2, 1]) * 180 / np.pi

In [ ]:
pol2cart(0, 1)